In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import pandas as pd
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir
from sklearn.preprocessing import scale
from pyclustertend import hopkins, vat, ivat
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/local/home/tsainbur/github_repos/avgn_paper/avgn/utils/general.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
from avgn.clusterability.hopkins import hopkins_statistic

In [5]:
datasets = list(DATA_DIR.glob('embeddings/*/*.pickle'))
len(datasets)

98

In [12]:
datasets

[PosixPath('/local/home/tsainbur/github_repos/avgn_paper/data/embeddings/castellucci_mouse_usv_segmented/VOC592.pickle'),
 PosixPath('/local/home/tsainbur/github_repos/avgn_paper/data/embeddings/castellucci_mouse_usv_segmented/VOC591.pickle'),
 PosixPath('/local/home/tsainbur/github_repos/avgn_paper/data/embeddings/castellucci_mouse_usv_segmented/VOC597.pickle'),
 PosixPath('/local/home/tsainbur/github_repos/avgn_paper/data/embeddings/castellucci_mouse_usv_segmented/VOC594.pickle'),
 PosixPath('/local/home/tsainbur/github_repos/avgn_paper/data/embeddings/BIRD_DB_Vireo_cassinii/full.pickle'),
 PosixPath('/local/home/tsainbur/github_repos/avgn_paper/data/embeddings/gibbon_morita_segmented/full.pickle'),
 PosixPath('/local/home/tsainbur/github_repos/avgn_paper/data/embeddings/bengalese_finch_sober/or60yw70.pickle'),
 PosixPath('/local/home/tsainbur/github_repos/avgn_paper/data/embeddings/bengalese_finch_sober/gr41rd51.pickle'),
 PosixPath('/local/home/tsainbur/github_repos/avgn_paper/data

In [6]:
def remove_outliers(z, pct = 99.5):
    """ GPU based UMAP algorithm produces some outliers that UMAP does not, but is much faster
    this is a quick fix for that. 
    """
    _min = np.percentile(z, (100-pct), axis=0)
    _max = np.percentile(z, pct, axis=0)
    for col in range(np.shape(z)[1]):
        mask = z[:,col] < _min[col]
        z[mask,col] = _min[col]
        mask = z[:,col] > _max[col]
        z[mask,col] = _max[col]
    return z

In [7]:
def gen_clusterability_df(dataset):

    save_loc = DATA_DIR / 'clusterability' / 'convex_sample' / \
        (dataset.parent.stem + '_ ' + dataset.stem + '.pickle')

    if save_loc.exists():
        return

    ds = pd.read_pickle(dataset)
    specs = np.stack(ds[['spectrogram']].spectrogram.values)
    specs = specs.reshape(len(specs), -1)
    specs = scale(specs)

    umap_proj = np.vstack(ds[['umap']].umap.values)
    umap_proj = remove_outliers(umap_proj, pct=99.5)
    umap_proj = scale(umap_proj)

    print((dataset, np.shape(umap_proj)))
    fig, ax = plt.subplots()
    ax.scatter(umap_proj[:, 0], umap_proj[:, 1], s=1, color='k', alpha=0.1)
    plt.show()

    hopkins_dict = {
        'umap':
        {
            0.01: hopkins_statistic(umap_proj, m_prop_n=0.01, n_neighbors=1, distribution="uniform_convex_hull"),
            0.1: hopkins_statistic(umap_proj, m_prop_n=0.1, n_neighbors=1, distribution="uniform_convex_hull"),
        },

    }

    clusterability_df = pd.DataFrame([[dataset, dataset.parent.stem, dataset.stem,
                                       hopkins_dict['umap'][0.01], hopkins_dict['umap'][0.1], ]], columns=[
        'df', 'dataset', 'indv',
        'umap_hopkins_1', 'umap_hopkins_10'
    ])

    clusterability_df.to_pickle(save_loc)

In [8]:
ensure_dir(DATA_DIR / 'clusterability' / 'convex_sample')

In [9]:
from joblib import Parallel, delayed

In [11]:
clust_data = Parallel(n_jobs=-1, verbose=10)(
    delayed(gen_clusterability_df)(dataset)
    for dataset in tqdm(datasets)
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    3.1s


[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  61 out of  98 | elapsed:    3.3s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  71 out of  98 | elapsed:    3.5s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  81 out of  98 | elapsed:    3.5s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  91 out of  98 | elapsed:    3.7s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  98 out of  98 | elapsed:    4.1s finished
